# SQL

Цель - проанализировать базу данных.
В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

## Подключение к базе данных

In [1]:
# импортируем библиотеки
!pip install SQLAlchemy

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import pandas as pd
from sqlalchemy import create_engine
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
            'pwd': 'Sdf4$2;d-d30pp', # пароль
            'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
            'port': 6432, # порт подключения
            'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                         db_config['pwd'],
                                                         db_config['host'],
                                                         db_config['port'],
                                                         db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

In [3]:
query = ''' SELECT *
            FROM books
        '''
books=pd.io.sql.read_sql(query, con=engine)
books.head()

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [4]:
query = ''' SELECT *
            FROM authors
        '''
authors=pd.io.sql.read_sql(query, con=engine)
authors.head()

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [5]:

query = ''' SELECT *
            FROM publishers
        '''
publishers=pd.io.sql.read_sql(query, con=engine)
publishers.head()

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [6]:
query = ''' SELECT *
            FROM ratings
        '''
ratings=pd.io.sql.read_sql(query, con=engine)
ratings.head()

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [7]:
query = ''' SELECT *
            FROM reviews
        '''
reviews=pd.io.sql.read_sql(query, con=engine)
reviews.head()

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


## Запросы

### Посчитайте, сколько книг вышло после 1 января 2000 года;

In [8]:
query = ''' SELECT COUNT(book_id) as cnt
            FROM books
            where publication_date >= '2000-01-01';
        '''
count_book=pd.io.sql.read_sql(query, con=engine, parse_dates=['publication_date'])
count_book

,cnt
0,821


### Для каждой книги посчитайте количество обзоров и среднюю оценку

In [9]:
query = ''' SELECT 
                    reviews.book_id as id_book,
                    count(reviews.review_id)as cnt_revievs,
                    AVG(ratings.rating) as avg_rating
            FROM reviews INNER JOIN ratings ON ratings.book_id = reviews.book_id
            GROUP BY id_book
            ORDER BY AVG(ratings.rating) DESC
        '''
book_mean_avg=pd.io.sql.read_sql(query, con=engine)#, parse_dates=['publication_date'])
book_mean_avg.head(15)

,id_book,cnt_revievs,avg_rating
0,55,4,5.0
1,518,4,5.0
2,347,6,5.0
3,513,4,5.0
4,57,4,5.0
5,642,4,5.0
6,459,4,5.0
7,972,4,5.0
8,182,4,5.0
9,418,4,5.0


In [10]:
book_mean_avg['cnt_revievs'].sum()

26167

In [11]:
query = ''' SELECT 
                    distinct books.title as title,
                    count(reviews.review_id)as cnt_revievs,
                    AVG(ratings.rating) as avg_rating
            FROM reviews INNER JOIN ratings ON ratings.book_id = reviews.book_id
                    inner join books ON books.book_id=reviews.book_id
            GROUP BY books.title
            ORDER BY AVG(ratings.rating) DESC
        '''

In [12]:
query = ''' SELECT 
                    books.book_id as id,
                    books.title as title,
                    count(distinct reviews.review_id)as cnt_revievs,
                    AVG(ratings.rating) as avg_rating
            FROM reviews left JOIN ratings ON ratings.book_id = reviews.book_id
                    left join books ON books.book_id=reviews.book_id
            GROUP BY books.book_id, books.title
            ORDER BY AVG(ratings.rating) DESC
        '''
book_mean_avg2=pd.io.sql.read_sql(query, con=engine)#, parse_dates=['publication_date'])
book_mean_avg2

,id,title,cnt_revievs,avg_rating
0,972,Wherever You Go There You Are: Mindfulness Me...,2,5.00
1,513,Piercing the Darkness (Darkness #2),2,5.00
2,901,The Walking Dead Book One (The Walking Dead #...,2,5.00
3,136,Captivating: Unveiling the Mystery of a Woman'...,2,5.00
4,86,Arrows of the Queen (Heralds of Valdemar #1),2,5.00
...,...,...,...,...
989,915,The World Is Flat: A Brief History of the Twen...,3,2.25
990,371,Junky,2,2.00
991,316,His Excellency: George Washington,2,2.00
992,202,Drowning Ruth,3,2.00


### Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;

In [13]:
query = ''' SELECT
                    count(books.book_id) as cnt_book,
                    publishers.publisher as publisher
            FROM books INNER JOIN publishers ON publishers.publisher_id = books.publisher_id
            where books.num_pages > 50
            group by publishers.publisher
            order by cnt_book desc
            limit 1
        '''
publisher_max_book=pd.io.sql.read_sql(query, con=engine)#, parse_dates=['publication_date'])
publisher_max_book

,cnt_book,publisher
0,42,Penguin Books


### Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;

In [14]:
query = ''' SELECT 
                    authors.author as author,
                    authors.author_id as id
            FROM authors
                INNER JOIN (
                    SELECT ROUND(AVG(ratings.rating),2) as book,
                    books.author_id as author_id
                    FROM 
                        books
                    LEFT JOIN ratings ON ratings.book_id=books.book_id
                        GROUP BY books.book_id
                    HAVING COUNT(ratings.username) > 50
                    ORDER BY 
                        AVG(ratings.rating) DESC
                        LIMIT 5
                        ) AS subq ON authors.author_id = subq.author_id;
        '''
max_rating_author=pd.io.sql.read_sql(query, con=engine)#, parse_dates=['publication_date'])
max_rating_author

,author,id
0,J.K. Rowling/Mary GrandPré,236
1,J.R.R. Tolkien,240
2,J.K. Rowling/Mary GrandPré,236
3,Markus Zusak/Cao Xuân Việt Khương,402
4,J.K. Rowling/Mary GrandPré,236


### Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

In [15]:
query = ''' select AVG(SUB.cnt_reviews) as mean_reviews
            FROM
            (SELECT (COUNT(review_id)) AS cnt_reviews
            FROM reviews
            WHERE username IN
                (
                    SELECT username
                    FROM ratings
                    GROUP BY username
                    HAVING COUNT(rating) > 50
                ) 
            GROUP BY username ) as SUB; 
        '''
temp = pd.io.sql.read_sql(query, con=engine)
temp

,mean_reviews
0,24.333333


## Вывод
После первого января 2020 годо вышла 821 книга. Издательство Penguin Books выпустило наибольшее число книг с более чем 50 страницами(42 книги), наибольшее число оценок у Джоан Роулинг(что не удивительно). среднее количество обзоров от пользователей, которые поставили больше 50 оценок - 24.